# **Q3. Test of association for N-dimensional tables**

# **Breast Cancer Data Description**

---



Citation Request:
   This breast cancer domain was obtained from the University Medical Centre,
   Institute of Oncology, Ljubljana, Yugoslavia.  Thanks go to M. Zwitter and 
   M. Soklic for providing the data.  Please include this citation if you plan
   to use this database.

1. Title: Breast cancer data (Michalski has used this)

2. Sources: 
   -- Matjaz Zwitter & Milan Soklic (physicians)
      Institute of Oncology 
      University Medical Center
      Ljubljana, Yugoslavia
   -- Donors: Ming Tan and Jeff Schlimmer (Jeffrey.Schlimmer@a.gp.cs.cmu.edu)
   -- Date: 11 July 1988

3. Past Usage: (Several: here are some)
     -- Michalski,R.S., Mozetic,I., Hong,J., & Lavrac,N. (1986). The 
      Multi-Purpose Incremental Learning System AQ15 and its Testing 
      Application to Three Medical Domains.  In Proceedings of the 
      Fifth National Conference on Artificial Intelligence, 1041-1045,
      Philadelphia, PA: Morgan Kaufmann.
      -- accuracy range: 66%-72%
     -- Clark,P. & Niblett,T. (1987). Induction in Noisy Domains.  In 
      Progress in Machine Learning (from the Proceedings of the 2nd
      European Working Session on Learning), 11-30, Bled, 
      Yugoslavia: Sigma Press.
      -- 8 test results given: 65%-72% accuracy range
     -- Tan, M., & Eshelman, L. (1988). Using weighted networks to 
      represent classification knowledge in noisy domains.  Proceedings 
      of the Fifth International Conference on Machine Learning, 121-134,
      Ann Arbor, MI.
      -- 4 systems tested: accuracy range was 68%-73.5%
    -- Cestnik,G., Konenenko,I, & Bratko,I. (1987). Assistant-86: A
      Knowledge-Elicitation Tool for Sophisticated Users.  In I.Bratko
      & N.Lavrac (Eds.) Progress in Machine Learning, 31-45, Sigma Press.
      -- Assistant-86: 78% accuracy

4. Relevant Information:
     This is one of three domains provided by the Oncology Institute
     that has repeatedly appeared in the machine learning literature.
     (See also lymphography and primary-tumor.)

     This data set includes 201 instances of one class and 85 instances of
     another class.  The instances are described by 9 attributes, some of
     which are linear and some are nominal.

5. Number of Instances: 286

6. Number of Attributes: 9 + the class attribute

7. Attribute Information:
   1. Class: no-recurrence-events, recurrence-events
   2. age: 10-19, 20-29, 30-39, 40-49, 50-59, 60-69, 70-79, 80-89, 90-99.
   3. menopause: lt40, ge40, premeno.
   4. tumor-size: 0-4, 5-9, 10-14, 15-19, 20-24, 25-29, 30-34, 35-39, 40-44, 45-49, 50-54, 55-59.
   5. inv-nodes: 0-2, 3-5, 6-8, 9-11, 12-14, 15-17, 18-20, 21-23, 24-26, 27-29, 30-32, 33-35, 36-39.
   6. node-caps: yes, no.
   7. deg-malig: 1, 2, 3.
   8. breast: left, right.
   9. breast-quad: left-up, left-low, right-up,	right-low, central.
  10. irradiat:	yes, no.

8. Missing Attribute Values: (denoted by "?")
   Attribute #:  Number of instances with missing values:
   6.             8
   9.             1.

9. Class Distribution:
    1. no-recurrence-events: 201 instances
    2. recurrence-events: 85 instances

In [1]:
#import the dataframe
import pandas as pd
import numpy as np
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/breast-cancer.data'
df = pd.read_csv(url, header=None)
categorical_features = ["Class","age","menopause","tumor-size","inv-nodes","node-caps","deg-malig","breast","breast-quad","irradiat"]
df.columns = categorical_features
df.head()

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no


In [0]:
# section 3

In [3]:
from scipy.stats import chi2_contingency

feature_num = len(categorical_features)
result = pd.DataFrame(index=categorical_features, columns=categorical_features)

def chi2_test(feature1,feature2):
    print("======================================================================================")
    print("                        Comparing ",categorical_features[i]," vs. ",categorical_features[j])
    print("======================================================================================")
    
    #Get the title of the two columns that we are intersted
    col_of_interest = []
    col_of_interest.append(feature1)
    col_of_interest.append(feature2)

    #create the sub dataframe with the selected two colums
    sub_df = df[col_of_interest].dropna()

    #drop entries where values are missing
    for k in col_of_interest:
      sub_df = sub_df[sub_df[k] != '?']

    #display the groupby count for two selected features
    stats_duo_features_sum_df_table = sub_df.groupby([feature1, feature2]).size().unstack(level=1, fill_value=0)
    
    #change all datatype to int
    stats_duo_features_sum_df_table = stats_duo_features_sum_df_table.astype('int32')

    print("\n" + "==>Summation Table:" + "\n")
    display(stats_duo_features_sum_df_table)

    #make hypophysis
    H0 = feature1 + ' and ' +  feature1 + ' are independent.'
    Ha = feature2 + ' and ' +  feature2 + ' are dependent.'

    #set the threshold of alpha
    alpha  = 0.05

    #Do the chi2 contingency calculation
    '''
    #The stat, pval, dof, exp_table = chi2_contingency(table) function Returns
    stat :The test statistic.
    pval: The p-value of the test
    dof : Degrees of freedom
    exp_table :The expected frequencies, based on the marginal sums of the table.
    '''
    stat, pval, dof, exp_table = chi2_contingency(stats_duo_features_sum_df_table.values)

    #compare the pval with the threshold of alpha, and display the result
    print("\n"+"==>Chi2 Contingency Test Resluts:"+"\n")
    if pval > 0.05:
      print('Accept null hypothesis.', H0)
      result.iloc[i,j] = "independent"
    else:
      print('Reject null hypothesis.', Ha)
      result.iloc[i,j] = "dependent"
    print("\n")


#loop to check each pair of features
for i in range(feature_num-1):
  for j in range(i+1,feature_num):
    chi2_test(categorical_features[i],categorical_features[j])    

                        Comparing  Class  vs.  age

==>Summation Table:



age,20-29,30-39,40-49,50-59,60-69,70-79
Class,,,,,,
no-recurrence-events,1,21,63,71,40,5
recurrence-events,0,15,27,25,17,1



==>Chi2 Contingency Test Resluts:

Accept null hypothesis. Class and Class are independent.


                        Comparing  Class  vs.  menopause

==>Summation Table:



menopause,ge40,lt40,premeno
Class,,,
no-recurrence-events,94,5,102
recurrence-events,35,2,48



==>Chi2 Contingency Test Resluts:

Accept null hypothesis. Class and Class are independent.


                        Comparing  Class  vs.  tumor-size

==>Summation Table:



...


==>Chi2 Contingency Test Resluts:

Reject null hypothesis. breast-quad and breast-quad are dependent.


                        Comparing  breast  vs.  irradiat

==>Summation Table:



irradiat,no,yes
breast,,
left,117,35
right,101,33



==>Chi2 Contingency Test Resluts:

Accept null hypothesis. breast and breast are independent.


                        Comparing  breast-quad  vs.  irradiat

==>Summation Table:



irradiat,no,yes
breast-quad,,
central,19,2
left_low,82,28
left_up,72,25
right_low,17,7
right_up,27,6



==>Chi2 Contingency Test Resluts:

Accept null hypothesis. breast-quad and breast-quad are independent.




In [4]:
#Summation of the test
display(result)

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
Class,NaN,independent,independent,independent,dependent,dependent,dependent,independent,independent,dependent
age,NaN,NaN,dependent,independent,independent,independent,independent,independent,independent,independent
menopause,NaN,NaN,NaN,independent,independent,independent,dependent,independent,independent,independent
tumor-size,NaN,NaN,NaN,NaN,independent,independent,dependent,independent,dependent,independent
inv-nodes,NaN,NaN,NaN,NaN,NaN,dependent,dependent,independent,independent,dependent
node-caps,NaN,NaN,NaN,NaN,NaN,NaN,dependent,independent,independent,dependent
deg-malig,NaN,NaN,NaN,NaN,NaN,NaN,NaN,independent,independent,dependent
breast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dependent,independent
breast-quad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,independent
irradiat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
